In [3]:
import pandas as pd

# path file Excel (upload dulu via Colab > files.upload())
file_path = "Copy of Dataset_-_Data_Penjualan_-_Kelompok_8(1).xlsx"

# load dataset
df = pd.read_excel(file_path)

# ambil sample 1500 baris (opsional, kalau datanya besar)
# kalau tidak mau sampling, langsung pakai df saja
df_sample = df.sample(n=1500, random_state=42) if len(df) > 1500 else df.copy()

# simpan hasil sampling sebagai data mentah tahap 1
df_sample.to_csv("data_raw.csv", index=False)

# tampilkan info dasar dataset
print("Jumlah baris dan kolom:", df_sample.shape)
print("\n5 data pertama:\n", df_sample.head())
print("\nJumlah missing tiap kolom:\n", df_sample.isnull().sum())

print("\nTahap 1 selesai! File data_raw.csv sudah dibuat.")


Jumlah baris dan kolom: (1500, 19)

5 data pertama:
       No.Faktur  Tgl Order                      Sales  \
654   VRD25HABKQ 2025-08-01    Abd Rahman S.T Prapani   
76    VRD25HAAGF 2025-08-01               Syarifuddin   
316   VRD25HAARV 2025-08-01  Alfino Hasan ( Prapani )   
1347  VRD25HACGH 2025-08-02   Arwan Galon ( Prapani )   
571   VRD25HABED 2025-08-01     Amiruddin ( Prapani )   

                           Pelanggan  Jumlah Bruto (Rp)  Total Discount (Rp)  \
654           FADILLAH, Tk - GOA RIA              46008                    0   
76             211, Tk - CENDRAWASIH             491040                    0   
316        IBIS MAKASSAR CITY CENTER            1670000                50100   
1347  MICHAEL KORS - MACCINI SOMBALA              55500                    0   
571         RAFA CELL - POROS MANDAI             153270                    0   

      Jumlah Netto (Rp)  Retur (Rp)   Total (Rp)   DiscD  DiscC      Tgl DO  \
654               46008            0        

In [4]:
import pandas as pd

# Load dataset hasil Tahap 1
df = pd.read_csv("data_raw.csv")

# --- Integrasi Data ---
# 1. Hapus data duplikat
df = df.drop_duplicates()

# 2. Normalisasi format huruf pada kolom 'Nama Produk'
if "Nama Produk" in df.columns:
    df["Nama Produk"] = df["Nama Produk"].str.strip().str.title()

# 3. Konversi kolom 'Tanggal' ke format datetime (jika ada)
if "Tanggal" in df.columns:
    df["Tanggal"] = pd.to_datetime(df["Tanggal"], errors="coerce")

# Simpan hasil integrasi
df.to_csv("data_integration.csv", index=False)

print("Tahap Integrasi selesai! File data_integration.csv sudah dibuat.")


Tahap Integrasi selesai! File data_integration.csv sudah dibuat.


In [15]:
import pandas as pd

# Load data integration
df_integration = pd.read_csv("data_integration.csv")

# cek 5 baris pertama untuk memastikan
print(df_integration.head())


   No.Faktur   Tgl Order                      Sales  \
0  VRD25HABKQ  2025-08-01    Abd Rahman S.T Prapani   
1  VRD25HAAGF  2025-08-01               Syarifuddin   
2  VRD25HAARV  2025-08-01  Alfino Hasan ( Prapani )   
3  VRD25HACGH  2025-08-02   Arwan Galon ( Prapani )   
4  VRD25HABED  2025-08-01     Amiruddin ( Prapani )   

                        Pelanggan  Jumlah Bruto (Rp)  Total Discount (Rp)  \
0          FADILLAH, Tk - GOA RIA              46008                    0   
1           211, Tk - CENDRAWASIH             491040                    0   
2       IBIS MAKASSAR CITY CENTER            1670000                50100   
3  MICHAEL KORS - MACCINI SOMBALA              55500                    0   
4        RAFA CELL - POROS MANDAI             153270                    0   

   Jumlah Netto (Rp)  Retur (Rp)   Total (Rp)   DiscD  DiscC       Tgl DO  \
0              46008            0        46008    0.0       0  2025-08-01   
1             491040            0       491040    0.

In [16]:
# --- Analisis Data ---

# Total transaksi
total_transaksi = len(df_integration)

# Total nilai penjualan
total_penjualan = df_integration["Total (Rp) "].sum()

# Rata-rata penjualan per transaksi
rata_penjualan = df_integration["Total (Rp) "].mean()

# Penjualan per bulan
df_integration["Tgl Order "] = pd.to_datetime(df_integration["Tgl Order "], errors="coerce")
penjualan_per_bulan = df_integration.groupby(df_integration["Tgl Order "].dt.to_period("M"))["Total (Rp) "].sum()

# Simpan hasil analisis ke file
with open("data_analysis.txt", "w") as f:
    f.write(f"Total transaksi: {total_transaksi}\n")
    f.write(f"Total nilai penjualan: Rp {total_penjualan:,.0f}\n")
    f.write(f"Rata-rata penjualan per transaksi: Rp {rata_penjualan:,.0f}\n")
    f.write("\nPenjualan per bulan:\n")
    f.write(str(penjualan_per_bulan))

print("Tahap Analisis selesai! Hasil disimpan di data_analysis.txt")


Tahap Analisis selesai! Hasil disimpan di data_analysis.txt


In [18]:
import pandas as pd

# === Tahap Validasi Data ===
# Load hasil dari integrasi
df_integration = pd.read_csv("data_integration.csv")

print("Jumlah data sebelum validasi:", len(df_integration))

# 1. Cek missing value
missing_values = df_integration.isnull().sum()
print("\nJumlah missing value per kolom:")
print(missing_values)

# 2. Hapus baris yang mengandung missing value
df_valid = df_integration.dropna()

# 3. Hapus duplikasi data
df_valid = df_valid.drop_duplicates()

# 4. Validasi tipe data (misal: kolom 'Jumlah' dan 'Harga' harus numerik)
if "Jumlah" in df_valid.columns:
    df_valid["Jumlah"] = pd.to_numeric(df_valid["Jumlah"], errors="coerce").fillna(0)

if "Harga" in df_valid.columns:
    df_valid["Harga"] = pd.to_numeric(df_valid["Harga"], errors="coerce").fillna(0)

# 5. Tambahkan kolom Total_Transaksi jika ada 'Jumlah' dan 'Harga'
if "Jumlah" in df_valid.columns and "Harga" in df_valid.columns:
    df_valid["Total_Transaksi"] = df_valid["Jumlah"] * df_valid["Harga"]

print("\nJumlah data setelah validasi:", len(df_valid))

# Simpan hasil validasi ke file CSV
df_valid.to_csv("data_validasi.csv", index=False)
print("Tahap Validasi selesai! File data_validasi.csv berhasil disimpan.")


Jumlah data sebelum validasi: 1500

Jumlah missing value per kolom:
No.Faktur                 0
Tgl Order                 0
Sales                     0
Pelanggan                 0
Jumlah Bruto (Rp)         0
Total Discount (Rp)       0
Jumlah Netto (Rp)         0
Retur (Rp)                0
Total (Rp)                0
DiscD                     0
DiscC                     0
Tgl DO                    0
No DO                     0
Drive                     0
Penagih                1392
Status                  497
Flag                    834
Tgl FP                   50
No FP                    50
dtype: int64

Jumlah data setelah validasi: 19
Tahap Validasi selesai! File data_validasi.csv berhasil disimpan.
